In [52]:
import pandas as pd
data=pd.read_csv('mid.csv')
data.head()


,question,answer
0,अंग्रेजी संसद में भारत सरकार अधिनियम कब पारित ...,[1858]
1,भारतीय राष्ट्रीय कांग्रेस का 1896 का अधिवेशन न...,[पहली बार राष्ट्रीय गीत गाया गया।]
2,निम्नलिखित में से किस गवर्नर जनरल को भारतीय प्...,सर चार्ल्स मेटकाफ
3,समुद्री व्यापार के सन्दर्भ में कार्ताज़ व्यवस्थ...,[पुर्तगाली ]
4,सुरेद्रनाथ बनर्जी के बारे में निम्नलिखित में क...,[उन्होंने आनंदमोहन बोस के साथ मिलकर साधारण ब्र...


In [53]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
    test_data = [
   "किस  गवर्नर जनरल को भारतीय प्रेस का मुक्तिदाता कहा जाता है?",
    "सुरेद्रनाथ बनर्जी के बारे में कुछ जानकारी है?",
    "समुद्री व्यापार के सन्दर्भ में कार्ताज़ व्यवस्था का उपयोग कैसे किया जाता था?",
    "अंग्रेजी संसद में भारत सरकार अधिनियम किस साल में पारित हुआ?",
    "भारतीय राष्ट्रीय कांग्रेस का 1896 का अधिवेशन किसके लिए विख्यात है?"
    
]
    


In [54]:
import numpy as np
def ratio(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [55]:

def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["question"], q,ratio_calc = True)
        if score >= 0.9: # I'm sure, stop here
            return row["answer"], score, row["question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["answer"]
            prediction = row["question"]
    if max_score > 0.5:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,किस गवर्नर जनरल को भारतीय प्रेस का मुक्तिदाता...,निम्नलिखित में से किस गवर्नर जनरल को भारतीय प्...,सर चार्ल्स मेटकाफ,0.859259
1,सुरेद्रनाथ बनर्जी के बारे में कुछ जानकारी है?,सुरेद्रनाथ बनर्जी के बारे में निम्नलिखित में क...,[उन्होंने आनंदमोहन बोस के साथ मिलकर साधारण ब्र...,0.690265
2,समुद्री व्यापार के सन्दर्भ में कार्ताज़ व्यवस्थ...,समुद्री व्यापार के सन्दर्भ में कार्ताज़ व्यवस्थ...,[पुर्तगाली ],0.936709
3,अंग्रेजी संसद में भारत सरकार अधिनियम किस साल म...,अंग्रेजी संसद में भारत सरकार अधिनियम कब पारित ...,[1858],0.899083
4,भारतीय राष्ट्रीय कांग्रेस का 1896 का अधिवेशन क...,भारतीय राष्ट्रीय कांग्रेस का 1896 का अधिवेशन न...,[पहली बार राष्ट्रीय गीत गाया गया।],0.897959
